<a href="https://colab.research.google.com/github/fcolombo7/AN2DL-2020/blob/main/Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration of the dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
from zipfile import ZipFile
import sys
sys.path.append('/content/drive/MyDrive/ANN_project2/starting_kit')

import prepare_submission
import read_mask_example
import decode_rle_example

import tensorflow as tf
import numpy as np

In [7]:
# Set the seed for random operations. 
SEED = 1234
tf.random.set_seed(SEED)

In [ ]:
#Definitions of constants

## Load the dataset

In [8]:
if not os.path.exists('Development_Dataset'):
  with ZipFile('./drive/MyDrive/ANN_project2/dataset.zip', 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall()
print('The dataset is loaded.')

The dataset is loaded.
